In [ ]:
from langchain_core.messages.ai import AIMessage
from langchain_core.messages.human import HumanMessage

In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_message_histories import ChatMessageHistory

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        ("placeholder", "{messages}"),
    ]
)

llm = ChatOllama(model="llama3", base_url="http://127.0.0.1:11434", keep_alive=-1)

chain = prompt | llm

In [ ]:
demo_ephemeral_chat_history = ChatMessageHistory()

In [ ]:
input1 = "Translate this sentence from English to French: I love programming."

demo_ephemeral_chat_history.add_user_message(input1)

response = chain.invoke(
    {
        "messages": demo_ephemeral_chat_history.messages,
    }
)

demo_ephemeral_chat_history.add_ai_message(response)
print(response.content)

In [ ]:
input2 = "What did I just ask you?"

demo_ephemeral_chat_history.add_user_message(input2)

response = chain.invoke(
    {
        "messages": demo_ephemeral_chat_history.messages,
    }
)

demo_ephemeral_chat_history.add_ai_message(response)
print(demo_ephemeral_chat_history)
print(response.content)

In [ ]:
demo_ephemeral_chat_history.messages

In [ ]:
for message in demo_ephemeral_chat_history.messages:
    if isinstance(message, AIMessage):
        print(f"Model: {message.response_metadata['model']}")
        print(f"Response Time: {message.response_metadata['created_at']}")

In [ ]:
message_history_dict = []

# Iterate through the messages in the chat history
for message in demo_ephemeral_chat_history.messages:
    if isinstance(message, HumanMessage):
        # Append the human message to the list
        message_history_dict.append({"type": "Human", "content": message.content})
    elif isinstance(message, AIMessage):
        # Append the AI message to the list
        message_history_dict.append({"type": "AI", "content": message.content})

# The resulting list of dictionaries
print(message_history_dict[0])

In [ ]:
import json
import os
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

history_dir = "sessions"

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(llm, get_session_history)

In [ ]:
config = {"configurable": {"session_id": "abc2"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)

response.content

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

In [ ]:
#store[session_id].messages
type(store['abc2'].messages)

In [ ]:
import json

messages = []
history = store['abc2']
for msg in history.messages:
    if isinstance(msg, HumanMessage):
        messages.append({"role": "human", "content": msg.content})
    elif isinstance(msg, AIMessage):
        messages.append({"role": "ai", "content": msg.content})
with open("sample.json", "w") as outfile:
    json.dump(messages, outfile)
messages

In [ ]:
with open("sample.json", "r") as readfile:
    msg_hist = json.load(readfile)
msg_hist

In [ ]:
print(msg_hist[0]['content'])
msg_hist

In [ ]:
ai_msg_hist = ChatMessageHistory()

for msg in msg_hist:
    if msg['role'] == 'human':
        ai_msg_hist.add_user_message(msg['content'])
    elif msg['role'] == 'ai':
        ai_msg_hist.add_ai_message(msg['content'])
ai_msg_hist

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="Oh, I forgot it again! What was my name?")],
    config=config,
)
response.content

In [ ]:
from session_handler import get_session_history

ses_id = "2"
msgs = get_session_history(ses_id)
msgs

In [ ]:
from chatbot_chain import get_chatbot_chain
from langchain_core.runnables.history import RunnableWithMessageHistory
from session_handler import get_session_history

chatbot_chain = get_chatbot_chain()
conversational_chatbot = RunnableWithMessageHistory(
    chatbot_chain, get_session_history
)

In [ ]:


config = {"configurable": {"session_id": "abc3"}}

response = conversational_chatbot.invoke(
    [HumanMessage(content="Hi! I'm Bob. What's yours?")],
    config=config,
)

response.content

In [ ]:
response = conversational_chatbot.invoke(
    [HumanMessage(content="Hey Benedict, do you know why you are here?")],
    config=config,
)

response.content

In [ ]:
from langchain_community.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults()

result = search.invoke("Trump")

In [ ]:
print(result)

In [ ]:
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchResults, DuckDuckGoSearchRun

@tool
def search_web(question):
    """
    This function utilizes the DuckDuckGoSearchResults class to perform a web search based on 
    the input question. It invokes the search and returns the results obtained from DuckDuckGo.

    Args:
        question: The query string or question to search on the web from user.

    Returns:
        str: A str of search results retrieved from DuckDuckGo.
        
    Usage Context:
        Use this tool to search the web for something you don't know about.
    """
    search = DuckDuckGoSearchResults()
    result = search.invoke(question)
    
    #return search
    return result

In [ ]:
question = "Who is Trump?"
result = search_web(question)

In [ ]:
print(result)

In [ ]:
type(result)

In [1]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.tools import Tool

web_search = DuckDuckGoSearchRun()
web_tool = Tool(
    name="web-search",
    description="Useful for when you need to do a search on the internet to find information that another tool can't find. be specific with your input.",
    func=web_search.run
)
tools = [web_tool]

In [ ]:
tool_names = []
for tool in tools:
    tool_names.append(tool.name)
    print(tool.name)

In [ ]:
from langchain import hub
prompt1 = hub.pull("hwchase17/react")
prompt1

In [ ]:
from langchain_core.prompts import PromptTemplate
agent_prompt = PromptTemplate.from_template("""
        Answer the following questions as best you can. You have access to the following tools:
        {tools}

        Use the following format:
        
        Question: the input question you must answer
        Thought: you should always think about what to do
        Action: the action to take, should be one of [{tool_names}]
        Action Input: the input to the action
        Observation: the result of the action
        ... (this Thought/Action/Action Input/Observation can repeat N times)
        Thought: I now know the final answer
        Final Answer: the final answer to the original input question

        Begin!

        Question: {input}
        Thought:{agent_scratchpad}
    """)
print(agent_prompt)

In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain.agents import AgentExecutor, create_react_agent
def get_chatbot_chain():
    agent_prompt = PromptTemplate.from_template("""
        Answer the following questions as best you can. You have access to the following tools:
        {tools}

        Use the following format:
        
        Question: the input question you must answer
        Thought: you should always think about what to do
        Action: the action to take, should be one of [{tool_names}]
        Action Input: the input to the action
        Observation: the result of the action
        ... (this Thought/Action/Action Input/Observation can repeat N times)
        Thought: I now know the final answer
        Final Answer: the final answer to the original input question

        Begin!

        Question: {input}
        Thought:{agent_scratchpad}
    """)

    llm = ChatOllama(model="llama3.1", base_url="http://127.0.0.1:11434", keep_alive=-1)
    
    agent = create_react_agent(llm, tools, agent_prompt)
    
    agent_exec = AgentExecutor(agent=agent, tools=tools, verbose=True, max_iterations=5, handle_parsing_errors=True)
    
    return agent_exec
chatbot_chain = get_chatbot_chain()

In [ ]:
question = {"input": "Hi, what's your name, how are you?"}
result = chatbot_chain.invoke(question)

In [ ]:
print(result['output'])

In [9]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_ollama import ChatOllama
def get_chatbot_chain():
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """
                    Your name is Benedict. You are a helpful assistant with access to a web search tool. 
                    You can use this tool to search the web using DuckDuckGo. Answer all questions to the best of your ability.
                    If you need additional information, use the web_tool to search for it.
                """,
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )

    llm = ChatOllama(model="llama3.1", base_url="http://127.0.0.1:11434", keep_alive=-1)
    
    llm_with_tools = llm.bind_tools(tools)

    chain = llm_with_tools | prompt
    
    return chain
chatbot_chain = get_chatbot_chain()

In [10]:
question = "Hi, what's your name, how are you?"
result = chatbot_chain.invoke(question)

ValueError: variable messages should be a list of base messages, got content='' response_metadata={'model': 'llama3.1', 'created_at': '2024-08-08T16:09:47.8404144Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'web-search', 'arguments': {'config': {'args': ['search for name and current state in English'], 'kwargs': {}}, 'kwargs': {}}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 1853502000, 'load_duration': 18450200, 'prompt_eval_count': 194, 'prompt_eval_duration': 300532000, 'eval_count': 93, 'eval_duration': 1533474000} id='run-c2281ebb-6fa9-4753-8491-a7a2f7620ce8-0' tool_calls=[{'name': 'web-search', 'args': {'config': {'args': ['search for name and current state in English'], 'kwargs': {}}, 'kwargs': {}}, 'id': '07f45f59-e317-41cc-acb7-e2cfe3c6f9bd', 'type': 'tool_call'}] usage_metadata={'input_tokens': 194, 'output_tokens': 93, 'total_tokens': 287} of type <class 'langchain_core.messages.ai.AIMessage'>